# Importing necessary libraries

In [39]:
import pandas as pd 
import numpy as np
import regex as re
import seaborn as sns
import matplotlib.pyplot as plt
import emoji
from autocorrect import Speller
from wordcloud import WordCloud
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer,WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,f1_score,make_scorer
from sklearn.compose import ColumnTransformer
import pickle
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


In [40]:
#Import dataset
data=pd.read_csv(r"C:\Users\Afrin\Downloads\reviews_data_dump\reviews_badminton\data.csv")",usecols=["Review text","Ratings"])

In [41]:
df=data.copy()

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8518 entries, 0 to 8517
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review text  8510 non-null   object
 1   Ratings      8518 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.2+ KB


In [43]:
df.isnull().sum()

Review text    8
Ratings        0
dtype: int64

In [44]:
df.dropna(inplace=True)

In [45]:
df.isnull().sum()

Review text    0
Ratings        0
dtype: int64

In [8]:
df.duplicated().sum()

3269

In [9]:
df["Ratings"].value_counts()

Ratings
5    5079
4    1744
1     766
3     615
2     306
Name: count, dtype: int64

In [10]:
def edat(data,name): 
    case=" ".join(data[name]).islower()
    html_tags=data[name].apply(lambda x:True if re.search("<.+?>",x) else False).sum()
    url=data[name].apply(lambda x:True if re.search("http[s]?://.+? +",x) else False).sum()
    unwanted_chars=data[name].apply(lambda x:True if re.search("[]()*\-:;\".,@#$%^&0-9]",x) else False).sum()
    if case==False:
        print("not in lower case")
    if html_tags>0:
        print("have html tags")
    if url>0:
        print("have urls")
    if unwanted_chars>0:
        print("have unwanted characters")

In [11]:
edat(df,"Review text")

not in lower case
have unwanted characters


In [12]:
stp=stopwords.words("english")
stp.remove("not")

In [13]:
fea_var=df.iloc[:,0]
cl_var=df.iloc[:,-1]

In [14]:
cl_var = cl_var.map(lambda x: "Negative" if x in [1, 2] else ("Positive" if x in [3, 4, 5] else x))

In [15]:
cl_var

0       Positive
1       Negative
2       Negative
3       Positive
4       Negative
          ...   
8505    Positive
8506    Positive
8507    Positive
8508    Positive
8509    Negative
Name: Ratings, Length: 8510, dtype: object

In [16]:
cl_var.value_counts()

Ratings
Positive    7438
Negative    1072
Name: count, dtype: int64

In [17]:
# splitting the data into xtrain and x_test
x_train,x_test,y_train,y_test=train_test_split(fea_var,cl_var,test_size=0.2,random_state=1,stratify=cl_var)

In [18]:
x_train=x_train.str.replace("READ MORE","")

In [19]:
x_test=x_test.str.replace("READ MORE","")

In [20]:
def lower(x):
    return x.str.lower()

In [21]:
def textpp(x,correct,emojii):
    if emojii==True:
        x=emoji.demojize(x)
    x=re.sub("<.+?>"," ",x)
    x=re.sub("http[s]?://.+? +"," ",x)
    x=re.sub("[]()!_*\-:;\".,@#$%^&0-9]"," ",x)

    if correct=="t":
        x=TextBlob(x).correct().string
    else:
        x=x

    return x

In [22]:
def pre_pro(x):
    return x.apply(textpp,args=("s",True))

In [23]:
def advpp(x,stemm): ## tokenization,removing stopwords,stemming
    ps=PorterStemmer()
    ls=LancasterStemmer()
    ss=SnowballStemmer(language="english")
    wl=WordNetLemmatizer()

    l=[]
    for word in word_tokenize(x):
        if word in stp:
            pass
        else:
            if stemm=="p":
                l.append(ps.stem(word))
            elif stemm=="l":
                l.append(ls.stem(word))
            elif stemm=="s":
                l.append(ss.stem(word))
            elif stemm=="lemma":
                l.append(wl.lemmatize(word,pos="v"))
            else:
                l.append(word)

    return " ".join(l)

In [24]:
def stw_advpp(x):
    return x.apply(lambda x : advpp(x,"lemma"))

In [59]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("sentiment_analysis")

2024/03/27 19:46:13 INFO mlflow.tracking.fluent: Experiment with name 'sentiment_analysis' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/302555786387097935', creation_time=1711548973861, experiment_id='302555786387097935', last_update_time=1711548973861, lifecycle_stage='active', name='sentiment_analysis', tags={}>

### creating pipeline for preprocessing

In [60]:
# pipe line for preprocessing. removing html tags,urls,unwanted characters and converting into lower case,demozije and stemmimg
pre_pro_pip=Pipeline([("lower",FunctionTransformer(lower)),
    ("preprocessing",FunctionTransformer(pre_pro)),
                     ("stp_stemmimg",FunctionTransformer(stw_advpp))])

In [61]:
pre_pro_pip

Pipeline(steps=[('lower',
                 FunctionTransformer(func=<function lower at 0x0000019E4375E3E0>)),
                ('preprocessing',
                 FunctionTransformer(func=<function pre_pro at 0x0000019E4375E020>)),
                ('stp_stemmimg',
                 FunctionTransformer(func=<function stw_advpp at 0x0000019E4C4FEAC0>))])

In [62]:
# transformed the x_train data which is learned in pipeline
fx_train=pre_pro_pip.fit_transform(x_train)

In [63]:
fx_test=pre_pro_pip.transform(x_test)

In [64]:
fy_train=y_train.map({"Positive":1,"Negative":0})

In [65]:
fy_test=y_test.map({"Positive":1,"Negative":0})

## model creation / training

In [66]:
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()
naive_bayes = MultinomialNB()
logistic_regression = LogisticRegression()
svm = SVC()
random_forest = RandomForestClassifier()

In [67]:
# Create pipelines
pipelines = {
    "Naive Bayes": Pipeline([
        ("vectorizer", count_vectorizer),
        ("classifier", naive_bayes)
    ]),
    "Logistic Regression": Pipeline([
        ("vectorizer", count_vectorizer),
        ("classifier", logistic_regression)
    ]),
    "SVM": Pipeline([
        ("vectorizer", count_vectorizer),
        ("classifier", svm)
    ]),
    "Random Forest": Pipeline([
        ("vectorizer", tfidf_vectorizer),
        ("classifier", random_forest)
    ])
}

# Grid search parameters
param_grids = {
    "Naive Bayes": {"classifier__alpha": [1, 5, 10, 15]},
    "Logistic Regression": {"classifier__C": [0.01, 0.1, 1, 10, 100],"classifier__penalty": ["l1", "l2"],
                            "classifier__solver": ["saga"]},
    "SVM": {"classifier__C": [0.01, 0.1, 1, 10, 100], "classifier__kernel": ["linear", "rbf"]},
    "Random Forest": {"classifier__n_estimators": [25, 50, 75, 100]}
}

# Define the scorer
scorer = make_scorer(f1_score, average='binary')

best_models = {}

# Grid Search with pipelines
for name, pipeline in pipelines.items():
    print("*"*10, name , "*"*10)
    grid_search = GridSearchCV(pipeline, param_grid=param_grids[name], cv=3, scoring=scorer, return_train_score=True)
    
    mlflow.sklearn.autolog(max_tuning_runs=None)
    with mlflow.start_run() as run:
      %time grid_search.fit(fx_train, fy_train)
   
    best_model = grid_search.best_estimator_
    best_models[name] = {
        "best_params": grid_search.best_params_,
        "best_score": grid_search.best_score_,
        "best_estimator": best_model 
    }

    # Evaluation
    y_pred = best_model.predict(fx_test)
    f1 = f1_score(fy_test, y_pred)
    print(f"{name} - F1 Score: {f1}")

# Print best parameters and scores for each model
for name, model_info in best_models.items():
    print("*"*10, name , "*"*10)
    print(f"{name} - Best Parameters: {model_info['best_params']}, Best F1 Score: {model_info['best_score']}")

2024/03/27 19:46:31 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


********** Naive Bayes **********
CPU times: total: 2.33 s
Wall time: 8.33 s
Naive Bayes - F1 Score: 0.9584158415841584
********** Logistic Regression **********


2024/03/27 19:46:40 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


CPU times: total: 16.7 s
Wall time: 35.3 s
Logistic Regression - F1 Score: 0.9503592423252776
********** SVM **********


2024/03/27 19:47:15 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


CPU times: total: 36.2 s
Wall time: 1min 22s
SVM - F1 Score: 0.9639192320423701
********** Random Forest **********


2024/03/27 19:48:38 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


CPU times: total: 35.9 s
Wall time: 1min 7s
Random Forest - F1 Score: 0.9642147117296223
********** Naive Bayes **********
Naive Bayes - Best Parameters: {'classifier__alpha': 1}, Best F1 Score: 0.9536032323990482
********** Logistic Regression **********
Logistic Regression - Best Parameters: {'classifier__C': 1, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}, Best F1 Score: 0.9496347704723629
********** SVM **********
SVM - Best Parameters: {'classifier__C': 10, 'classifier__kernel': 'rbf'}, Best F1 Score: 0.9572734476755228
********** Random Forest **********
Random Forest - Best Parameters: {'classifier__n_estimators': 100}, Best F1 Score: 0.9591910054227851


In [68]:
for name, model_info in best_models.items():
    model = model_info['best_estimator']
    filename = f"{name}_model.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

In [57]:
for name, model_info in best_models.items():
    print("*"*10, name, "*"*10)
    
    y_test_pred = model.predict(fx_test)
    print("Test Score (F1)",f1_score(fy_test, y_test_pred))
    

********** Naive Bayes **********
Test Score (F1) 0.9612454455117588
********** Logistic Regression **********
Test Score (F1) 0.9612454455117588
********** SVM **********
Test Score (F1) 0.9612454455117588
********** Random Forest **********
Test Score (F1) 0.9612454455117588


In [58]:
# Stop the auto logger

mlflow.sklearn.autolog(disable=True)